In [ ]:
# Author: QiangZiBro
# Time: 2021-10-14
# Contact: Github/QiangZiBro

# data analysis and wrangling
import pandas as pd
import numpy as np
import random as rnd
import warnings
# visualization
import seaborn as sns
import matplotlib.pyplot as plt

plt.style.use('ggplot')
# MacOS上画图中文乱码的问题
# https://blog.csdn.net/minixuezhen/article/details/81516949?utm_medium=distribute.pc_relevant.none-task-blog-
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS']

# %matplotlib inline

import sys
import os

from pprint import pprint

if not os.path.exists("results"):
    os.mkdir("results")

warnings.filterwarnings("ignore")
FIGSIZE = (30, 25)
FORMAT = "jpg"

## 数据加载工具

In [ ]:
PLACES = ['A', 'B', 'C', 'A1', 'A2', 'A3']
TYPES = [0, 1, 2]
def _load_data():
    table_files = """data/附件1 监测点A空气质量预报基础数据.xlsx
data/附件2 监测点B、C空气质量预报基础数据.xlsx
data/附件3 监测点A1、A2、A3空气质量预报基础数据.xlsx""".split("\n")
    tables = [pd.read_excel("../"+i, engine='openpyxl', sheet_name=None) for i in table_files] # 表格全读取
    keys = [list(i.keys()) for i in tables] # 每个表格的sheet list
    return tables, keys

def _process_to_json(tables):
    sheetnames = {0:'监测点{}逐小时污染物浓度与气象一次预报数据', 1:'监测点{}逐小时污染物浓度与气象实测数据', 2:'监测点{}逐日污染物浓度实测数据'}
    result = {i:{} for i in PLACES}
    
    for k,v in sheetnames.items():
        tables[0][v.format('A')].name = v.format('A')
        result['A'][k] = tables[0][v.format('A')]
    for c in ['B', 'C']:
        for k,v in sheetnames.items():
            tables[1][v.format(c)].name = v.format(c)
            result[c][k] = tables[1][v.format(c)]
    for c in ['A1', 'A2', 'A3']:
        for k,v in sheetnames.items():
            tables[2][v.format(c)].name = v.format(c)
            result[c][k] = tables[2][v.format(c)]
    return result

def load(): 
    """得到一个Json格式的数据项
    
    Return:
        {
            "A":{
                "0":DataFrame,
                "1":DataFrame,
                "2":DataFrame,
            }
            ...
        }
    """
    
    tables, keys = _load_data()
    return _process_to_json(tables)

data = load()

## 一键报告（仅供参考）

In [ ]:
import pandas_profiling
from pathlib import Path
for p in PLACES:
    for i in range(3):
        profile = data[p][i].profile_report(title = f"{p}_{i}")
        profile.to_file(output_file = Path(f"./{p}_{i}_report.html"))

In [ ]:
import sweetviz as sv
my_report = sv.analyze(data['A'][2])
my_report.show_html()

for p in PLACES:
    for i in range(3):
        my_report = sv.analyze(data[p][i])
        my_report.show_html(Path(f"./sv_{p}_{i}_report.html"))

In [ ]:
print(data.keys())
print(data['A'].keys())
# for p in PLACES:
#     for i in range(3): 
#         print(data[p][i].min())

# 数据处理
- 缺失值填充
- 负值替换

In [ ]:
def create_new_table_for_place(p):
    writer = pd.ExcelWriter("../data/" + p + '.xlsx', engine='xlsxwriter')
    for i in range(3):
        df = data[p][i]
        # 数值化
        index = list(df.columns).index('地点')+1
        df1 = df[df.columns[index:]]
        # 异常值处理
        df1 = df1.apply(pd.to_numeric, errors='coerce')
        # 负值处理
        num = df1._get_numeric_data()
        num[num < 0] = np.nan
        # 空值用中位数填充
        df1 = df1.fillna(df.median(numeric_only=True))
        df[df.columns[index:]] = df1
        df.to_excel(writer, sheet_name=str(i))
        
    writer.save()
for p in PLACES:
    create_new_table_for_place(p)

# create_new_table_for_place(PLACES[0])

In [ ]:
df = pd.read_excel("../data/{}.xlsx".format('A'), engine='openpyxl', sheet_name=None)
df = df['1']
df.isna().sum()

In [ ]:
df.fillna(df.mean()).iloc[500]

In [ ]:
df.iloc[500]

In [ ]:
# 数值化
index = list(df.columns).index('地点')+1
df1 = df[df.columns[index+1:]]
# 异常值处理
df1 = df1.apply(pd.to_numeric, errors='coerce')
# 负值处理
num = df1._get_numeric_data()
num[num < 0] = np.nan
# 空值用中位数填充
df1 = df1.fillna(df.median(numeric_only=True))
df[df.columns[index+1:]] = df1

In [ ]:
df.iloc[208]

In [ ]:
df.columns[index:]

## 预处理完分析

In [ ]:
df = data['A'][1]
index = list(data['A'][0].columns).index('地点')+1
df1 = df[df.columns[index+1:]]
# 异常值处理
df1 = df1.apply(pd.to_numeric, errors='coerce')
# 负值处理
num = df1._get_numeric_data()
num[num < 0] = np.nan
# 空值用中位数填充
df1 = df1.fillna(df.median(numeric_only=True))
df[df.columns[index+1:]] = df1
df.iloc[182]

In [ ]:
type(data['A'][0].columns)

In [ ]:
data['A'][0].columns

In [ ]:
list(data['A'][0].columns).index('地点')

In [ ]:
data['A'][0].columns.get_indexer_for

In [ ]:
df.iloc[182]

In [ ]:
df.apply(pd.to_numeric, errors='coerce')

In [ ]:
df["SO2监测浓度(μg/m³)"].isna().sum()

## 预处理完分析

In [ ]:
for p in PLACES:
    df = pd.read_excel("../data/{}.xlsx".format(p), engine='openpyxl', sheet_name=None)
    for i in range(3):
        num = df[str(i)]._get_numeric_data()
        if (num<0).sum().sum() != 0:
            print(p,i)

In [ ]:
df = pd.read_excel("../data/{}.xlsx".format('A'), engine='openpyxl', sheet_name=None)
df = df['1']
df.fillna(df.mean()).iloc[182]

In [ ]:
df.iloc[182]

In [ ]:
df

In [ ]:
df['监测点A1逐日污染物浓度实测数据'] = df['监测点A1逐日污染物浓度实测数据'].fillna(df['监测点A1逐日污染物浓度实测数据'].mean())

## 空值分析

In [ ]:
def null_values_analysis(data):
    NULL_COUNTS = {}
    for p in PLACES:
        for t in TYPES:  
            if data[p][t].isnull().sum().sum():
                NULL_COUNTS.update({p+"_"+str(t):data[p][t].isnull().sum()})
    return NULL_COUNTS

def null_values_analysis_plot(null_table):
    fig, axs = plt.subplots(3, 3, figsize=FIGSIZE)
    fig.tight_layout(h_pad=10)
    fig.suptitle('空值分析', size=20)
    
    for i, (k,v) in enumerate(null_table.items()):
        axs[i//3][i%3].bar(v.keys(), v, 0.35)
        axs[i//3][i%3].set_title(k)
        axs[i//3][i%3].set_xticklabels(labels=v.keys(),rotation=45)
        axs[i//3][i%3].grid(True)

    font = {'family': 'serif',
            'color':  'darkred',
            'weight': 'normal',
            'size': 16,
            }

    plt.savefig("results/nan_value_analysis."+FORMAT, format=FORMAT)
    plt.close()
null_table = null_values_analysis(data)
null_values_analysis_plot(null_table)

# 数据分析
下面首先对各量随时间的分布进行分析
注意数字和子表格的对应：
- 0 逐小时污染物浓度与气象一次预报数据
- 1 逐小时污染物浓度与气象实测数据
- 2 逐日污染物浓度实测数据

## 0 对逐小时污染物浓度与气象一次预报数据的分析

In [ ]:
key = data['A'][0].columns
key, len(key[3:])

In [ ]:
PREDICTED_KEYS = key[3:]
print("预测天气状况",PREDICTED_KEYS[:-6])
print("预测污染因素",PREDICTED_KEYS[-6:])

In [ ]:
def plot_prediction_hist_by_time(place):
    "对逐小时污染物浓度与气象一次预报数据的分析"
    key = data[place][0].columns
    PREDICTED_KEYS = key[3:]
    fig, axes = plt.subplots(nrows=7, ncols=3, figsize=FIGSIZE)
    fig.tight_layout(h_pad=3)
    plt.suptitle(place+"_0", size=30)
    
    for i, k in enumerate(PREDICTED_KEYS):
        data[place][0].plot.line(ax = axes[i//3][i%3], x = key[1] , y=k)
    plt.savefig("results/{}_0逐小时污染物浓度与气象一次预报数据.{}".format(place, FORMAT), format=FORMAT)
    plt.close()


for p in PLACES:
    plot_prediction_hist_by_time(p)

## 1 对逐小时污染物浓度与气象实测数据的分析

In [ ]:
key = data['A'][1].columns
key, len(key[2:])

In [ ]:
MEASURED_KEYS = key[2:]
print("测量天气状况",MEASURED_KEYS[:6])
print("测量污染因素",MEASURED_KEYS[6:])

In [ ]:
def plot_measured_hist_by_hour(place):
    "对逐小时污染物浓度与气象一次预报数据的分析"
    key = data[place][1].columns
    MEASURED_KEYS = key[2:]
    fig, axes = plt.subplots(nrows=4, ncols=3, figsize=FIGSIZE)
    for i, k in enumerate(MEASURED_KEYS):
        axes[i//3][i%3].set_xticks([])
        # 预处理
        data[place][1][k].fillna(0)
        # 将object类型转为数值类型
        data[place][1][k] = pd.to_numeric(data[place][1][k], errors='coerce')
        data[place][1].plot.line(ax = axes[i//3][i%3], x = key[0] , y=k)
    fig.tight_layout(h_pad=3)
    plt.suptitle(place+"_1", size=30)
    plt.savefig("results/{}_1逐小时污染物浓度与气象实测数据.{}".format(place, FORMAT), format=FORMAT)
    plt.close()
    
for p in PLACES:
    plot_measured_hist_by_hour(p)

## 2 对逐日污染物浓度实测数据的分析

In [ ]:
key = data['A'][2].columns
key, len(key[2:])

In [ ]:
MEASURED_KEYS = key[2:]
print("测量污染因素",MEASURED_KEYS[:])

In [ ]:
def plot_measured_polution_hist_by_day(place):
    "对逐小时污染物浓度与气象一次预报数据的分析"
    key = data[place][2].columns
    MEASURED_KEYS = key[2:]
    fig, axes = plt.subplots(nrows=2, ncols=3, figsize=FIGSIZE)
    for i, k in enumerate(MEASURED_KEYS):
        # 预处理
        data[place][2][k].fillna(0)
        # 将object类型转为数值类型
        data[place][2][k] = pd.to_numeric(data[place][2][k], errors='coerce')
        data[place][2].plot.line(ax = axes[i//3][i%3], x = key[0] , y=k)
    fig.tight_layout(h_pad=3)
    plt.suptitle(place+"_2", size=30)
    plt.savefig("results/{}_2逐日污染物浓度实测数据.{}".format(place, FORMAT), format=FORMAT)
    plt.close()
# 解锁运行  
for p in PLACES:
    plot_measured_polution_hist_by_day(p)